In [118]:
# !pip install --upgrade pip

import os

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import warnings
import random


pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
import seaborn as sns


# Global utitlity functions are in separate notebook
%run ./SEED-CONSTANTS.ipynb

SEED: 567
FINAL_EXP_RESULTS_PATH: clip=False_experiments_results
--------------------SEED and CONSTANTS imported--------------------


In [119]:
# FINAL_EXP_RESULTS_PATH="final_experiments_results"
FINAL_EXP_RESULTS_PATH#="clip=False_experiments_results"

CLASSIFIER_TYPE_LST=["OCSVM", "kNN", "LOF", "IF"]

TRAINING_CONFIG_DICT_FILE_NAME#="model_archi_performance_lr_dict.json"
TRAINING_CONFIG_DICT_FOLDER_PATH#="siamese_cnn_results_final"

'siamese_cnn_results_final'

In [88]:
# df = df.loc[df.groupby([("", "type")])[("", "mean rank"),].idxmin()[('', 'mean rank')]]
# df

In [89]:
df.insert(0, ("", "classifier"), "OCSVM")
df

ValueError: cannot insert ('', 'classifier'), already exists

In [141]:
top_eer_df_lst=[]
top_gini_df_lst=[]
for clf_name in CLASSIFIER_TYPE_LST:
    eer_df = pd.read_json(f"{FINAL_EXP_RESULTS_PATH}/WACA-{clf_name}/WACA-{clf_name}-df.json")
    gini_df = pd.read_json(f"{FINAL_EXP_RESULTS_PATH}/WACA-{clf_name}/WACA-{clf_name}-gini-df.json")
    
    idx_tuples = [eval(i) for i in eer_df.columns.tolist()]

    # both eer_df and gini_df have same columns
    # Convert list of tuples back to a MultiIndex
    multiindex_columns = pd.MultiIndex.from_tuples(idx_tuples)

    eer_df.columns = multiindex_columns
    top_eer_df = eer_df.loc[eer_df.groupby([("", "type")])[("", "mean rank"),].idxmin()[('', 'mean rank')]]
    top_eer_df.insert(0, ("", "classifier"), clf_name)
    
    top_eer_df_lst.append(top_eer_df)
    
    
    gini_df.columns = multiindex_columns
    top_gini_df = gini_df.loc[eer_df.groupby([("", "type")])[("", "mean rank"),].idxmin()[('', 'mean rank')]]
    top_gini_df.insert(0, ("", "classifier"), clf_name)
    
    top_gini_df_lst.append(top_gini_df)

# EER
top_clf_eer_df = pd.concat(top_eer_df_lst)
top_clf_eer_df.drop(("", 'mean rank'), axis=1, inplace=True)
top_clf_eer_df.reset_index(drop=True, inplace=True)

top_clf_eer_df.insert(0, ("", 'mean rank'), top_clf_eer_df[("", "mean")].rank(method='min').astype(np.int64))
top_clf_eer_df=top_clf_eer_df.sort_values(("", 'mean rank'))

# GINI
top_clf_gini_df = pd.concat(top_gini_df_lst)
top_clf_gini_df.drop(("", 'mean rank'), axis=1, inplace=True)
top_clf_gini_df.reset_index(drop=True, inplace=True)

top_clf_gini_df.insert(0, ("", 'mean rank'), top_clf_gini_df[("", "mean")].rank(method='min').astype(np.int64))
top_clf_gini_df=top_clf_gini_df.sort_values(("", 'mean rank'))


In [142]:
top_clf_eer_df

Window Size  \
   mean rank classifier cut_off_freq EMA_span SMA_winsize   type         125   
7          1        LOF            -       49           -  Naive    0.129547   
1          2      OCSVM           24       30           -  Naive    0.150753   
8          3        LOF           26        -           2   Real    0.154874   
4          4        kNN            -       37           -  Naive    0.151440   
6          5        LOF            -        -           -      -    0.159600   
2          6      OCSVM           26        -           2   Real    0.161919   
5          7        kNN           33        -           -   Real    0.181438   
0          8      OCSVM            -        -           -      -    0.164775   
3          9        kNN            -        -           -      -    0.193208   
10        10         IF            -       37           -  Naive    0.196879   
11        11         IF           49        -           -   Real    0.233231   
9         12         IF            -        -           -      -    0.239770   

                                                                          \
         250       500       750      1000      1250      1500      1750   
7   0.090563  0.062048  0.050370  0.041057  0.038831  0.033608  0.033575   
1   0.099805  0.064574  0.055185  0.043135  0.037153  0.032977  0.033688   
8   0.096507  0.064190  0.048822  0.042864  0.040104  0.037037  0.031965   
4   0.100876  0.066600  0.053737  0.044941  0.038657  0.033745  0.034783   
6   0.102140  0.066176  0.054276  0.046387  0.041725  0.035460  0.035910   
2   0.099279  0.064913  0.049414  0.045881  0.044907  0.036927  0.038084   
5   0.104868  0.068005  0.048114  0.043948  0.038368  0.041152  0.035910   
0   0.107527  0.067537  0.059697  0.048925  0.048507  0.038162  0.039565   
3   0.113839  0.067604  0.054209  0.048419  0.042245  0.040123  0.036634   
10  0.167937  0.141428  0.130236  0.116974  0.115694  0.123978  0.122625   
11  0.188434  0.155029  0.145380  0.140542  0.131343  0.128656  0.124348   
9   0.190747  0.164511  0.145838  0.144589  0.133275  0.134019  0.135185   

                                  
        2000      mean  variance  
7   0.034444  0.057116  0.001081  
1   0.036519  0.061532  0.001578  
8   0.040556  0.061880  0.001603  
4   0.036944  0.062414  0.001575  
6   0.037315  0.064332  0.001727  
2   0.042944  0.064919  0.001697  
5   0.039537  0.066816  0.002328  
0   0.039611  0.068256  0.001781  
3   0.039815  0.070677  0.002688  
10  0.117111  0.136985  0.000779  
11  0.125796  0.152529  0.001316  
9   0.134352  0.158032  0.001299

In [112]:
top_clf_eer_df.style.format(STYLER_ERR_FORMAT_DICT).hide(axis='index').to_latex()

'\\begin{tabular}{rlllllrrrrrrrrrrr}\n\\multicolumn{6}{r}{} & \\multicolumn{9}{r}{Window Size} & \\multicolumn{2}{r}{} \\\\\nmean rank & classifier & cut_off_freq & EMA_span & SMA_winsize & type & 125 & 250 & 500 & 750 & 1000 & 1250 & 1500 & 1750 & 2000 & mean & variance \\\\\n1 & LOF & - & 49 & - & Naive & 0.1295 & 0.0906 & 0.0620 & 0.0504 & 0.0411 & 0.0388 & 0.0336 & 0.0336 & 0.0344 & 0.0571 & 0.0011 \\\\\n2 & OCSVM & 24 & 30 & - & Naive & 0.1508 & 0.0998 & 0.0646 & 0.0552 & 0.0431 & 0.0372 & 0.0330 & 0.0337 & 0.0365 & 0.0615 & 0.0016 \\\\\n3 & LOF & 26 & - & 2 & Real & 0.1549 & 0.0965 & 0.0642 & 0.0488 & 0.0429 & 0.0401 & 0.0370 & 0.0320 & 0.0406 & 0.0619 & 0.0016 \\\\\n4 & kNN & - & 37 & - & Naive & 0.1514 & 0.1009 & 0.0666 & 0.0537 & 0.0449 & 0.0387 & 0.0337 & 0.0348 & 0.0369 & 0.0624 & 0.0016 \\\\\n5 & LOF & - & - & - & - & 0.1596 & 0.1021 & 0.0662 & 0.0543 & 0.0464 & 0.0417 & 0.0355 & 0.0359 & 0.0373 & 0.0643 & 0.0017 \\\\\n6 & OCSVM & 26 & - & 2 & Real & 0.1619 & 0.0993 & 0.064

$$
\begin{tabular}{rlllllrrrrrrrrrrr}
\multicolumn{6}{r}{} & \multicolumn{9}{r}{Window Size} & \multicolumn{2}{r}{} \\
mean rank & classifier & cut_off_freq & EMA_span & SMA_winsize & type & 125 & 250 & 500 & 750 & 1000 & 1250 & 1500 & 1750 & 2000 & mean & variance \\
1 & LOF & - & 49 & - & Naive & 0.1295 & 0.0906 & 0.0620 & 0.0504 & 0.0411 & 0.0388 & 0.0336 & 0.0336 & 0.0344 & 0.0571 & 0.0011 \\
2 & OCSVM & 24 & 30 & - & Naive & 0.1508 & 0.0998 & 0.0646 & 0.0552 & 0.0431 & 0.0372 & 0.0330 & 0.0337 & 0.0365 & 0.0615 & 0.0016 \\
3 & LOF & 26 & - & 2 & Real & 0.1549 & 0.0965 & 0.0642 & 0.0488 & 0.0429 & 0.0401 & 0.0370 & 0.0320 & 0.0406 & 0.0619 & 0.0016 \\
4 & kNN & - & 37 & - & Naive & 0.1514 & 0.1009 & 0.0666 & 0.0537 & 0.0449 & 0.0387 & 0.0337 & 0.0348 & 0.0369 & 0.0624 & 0.0016 \\
5 & LOF & - & - & - & - & 0.1596 & 0.1021 & 0.0662 & 0.0543 & 0.0464 & 0.0417 & 0.0355 & 0.0359 & 0.0373 & 0.0643 & 0.0017 \\
6 & OCSVM & 26 & - & 2 & Real & 0.1619 & 0.0993 & 0.0649 & 0.0494 & 0.0459 & 0.0449 & 0.0369 & 0.0381 & 0.0429 & 0.0649 & 0.0017 \\
7 & kNN & 33 & - & - & Real & 0.1814 & 0.1049 & 0.0680 & 0.0481 & 0.0439 & 0.0384 & 0.0412 & 0.0359 & 0.0395 & 0.0668 & 0.0023 \\
8 & OCSVM & - & - & - & - & 0.1648 & 0.1075 & 0.0675 & 0.0597 & 0.0489 & 0.0485 & 0.0382 & 0.0396 & 0.0396 & 0.0683 & 0.0018 \\
9 & kNN & - & - & - & - & 0.1932 & 0.1138 & 0.0676 & 0.0542 & 0.0484 & 0.0422 & 0.0401 & 0.0366 & 0.0398 & 0.0707 & 0.0027 \\
\end{tabular}
$$


In [143]:
top_clf_gini_df

Window Size  \
   mean rank classifier cut_off_freq EMA_span SMA_winsize   type         125   
9          1         IF            -        -           -      -    0.335525   
11         2         IF           49        -           -   Real    0.329424   
10         3         IF            -       37           -  Naive    0.411922   
2          4      OCSVM           26        -           2   Real    0.517772   
0          5      OCSVM            -        -           -      -    0.505542   
3          6        kNN            -        -           -      -    0.500338   
5          7        kNN           33        -           -   Real    0.500188   
4          8        kNN            -       37           -  Naive    0.590468   
6          9        LOF            -        -           -      -    0.535434   
1         10      OCSVM           24       30           -  Naive    0.585105   
8         11        LOF           26        -           2   Real    0.536458   
7         12        LOF            -       49           -  Naive    0.641991   

                                                                          \
         250       500       750      1000      1250      1500      1750   
9   0.382443  0.422797  0.444283  0.481188  0.491695  0.505200  0.497006   
11  0.370514  0.415564  0.441483  0.490492  0.509331  0.527388  0.549604   
10  0.420017  0.447742  0.479385  0.517618  0.514364  0.514173  0.500893   
2   0.643776  0.751348  0.803055  0.809384  0.829675  0.845695  0.831326   
0   0.648686  0.760365  0.802483  0.824444  0.837377  0.846027  0.853184   
3   0.643844  0.750055  0.803336  0.823235  0.840662  0.852607  0.863799   
5   0.637842  0.756474  0.803453  0.822859  0.842396  0.852752  0.860681   
4   0.680762  0.758776  0.798135  0.818997  0.831698  0.839841  0.861540   
6   0.669907  0.768247  0.812912  0.834429  0.838725  0.854134  0.869947   
1   0.681948  0.771622  0.795331  0.830572  0.835501  0.848243  0.856070   
8   0.673346  0.776024  0.818321  0.830838  0.840478  0.861742  0.871662   
7   0.711383  0.774322  0.806974  0.833123  0.833132  0.851602  0.870412   

                                  
        2000      mean  variance  
9   0.533526  0.454852  0.004147  
11  0.558650  0.465828  0.006592  
10  0.521120  0.480804  0.001895  
2   0.843836  0.763985  0.012590  
0   0.854115  0.770247  0.014135  
3   0.876040  0.772657  0.015597  
5   0.878800  0.772827  0.015805  
4   0.871769  0.783554  0.008660  
6   0.884366  0.785345  0.012973  
1   0.869509  0.785989  0.008959  
8   0.875588  0.787162  0.012772  
7   0.873434  0.799597  0.006092

In [113]:
top_clf_gini_df.style.format(STYLER_ERR_FORMAT_DICT).hide(axis='index').to_latex()

'\\begin{tabular}{rlllllrrrrrrrrrrr}\n\\multicolumn{6}{r}{} & \\multicolumn{9}{r}{Window Size} & \\multicolumn{2}{r}{} \\\\\nmean rank & classifier & cut_off_freq & EMA_span & SMA_winsize & type & 125 & 250 & 500 & 750 & 1000 & 1250 & 1500 & 1750 & 2000 & mean & variance \\\\\n1 & OCSVM & 26 & - & 2 & Real & 0.5178 & 0.6438 & 0.7513 & 0.8031 & 0.8094 & 0.8297 & 0.8457 & 0.8313 & 0.8438 & 0.7640 & 0.0126 \\\\\n2 & OCSVM & - & - & - & - & 0.5055 & 0.6487 & 0.7604 & 0.8025 & 0.8244 & 0.8374 & 0.8460 & 0.8532 & 0.8541 & 0.7702 & 0.0141 \\\\\n3 & kNN & - & - & - & - & 0.5003 & 0.6438 & 0.7501 & 0.8033 & 0.8232 & 0.8407 & 0.8526 & 0.8638 & 0.8760 & 0.7727 & 0.0156 \\\\\n4 & kNN & 33 & - & - & Real & 0.5002 & 0.6378 & 0.7565 & 0.8035 & 0.8229 & 0.8424 & 0.8528 & 0.8607 & 0.8788 & 0.7728 & 0.0158 \\\\\n5 & kNN & - & 37 & - & Naive & 0.5905 & 0.6808 & 0.7588 & 0.7981 & 0.8190 & 0.8317 & 0.8398 & 0.8615 & 0.8718 & 0.7836 & 0.0087 \\\\\n6 & LOF & - & - & - & - & 0.5354 & 0.6699 & 0.7682 & 0.8129 

In [145]:
top_eer_df_lst=[]
top_gini_df_lst=[]
for clf_name in CLASSIFIER_TYPE_LST:
    # print(clf_name)
    eer_df = pd.read_json(f"{FINAL_EXP_RESULTS_PATH}/WACA-{clf_name}/WACA-{clf_name}-df.json")
    gini_df = pd.read_json(f"{FINAL_EXP_RESULTS_PATH}/WACA-{clf_name}/WACA-{clf_name}-gini-df.json")
    
    idx_tuples = [eval(i) for i in eer_df.columns.tolist()]

    # both eer_df and gini_df have same columns
    # Convert list of tuples back to a MultiIndex
    multiindex_columns = pd.MultiIndex.from_tuples(idx_tuples)

    eer_df.columns = multiindex_columns

    top_eer_df = eer_df.loc[eer_df.groupby([("", "type")])[("", "mean rank"),].idxmin()[('', 'mean rank')]]
    top_eer_df.insert(0, ("", "classifier"), clf_name)
    
    top_eer_df_lst.append(top_eer_df)
    
    
    # idx_tuples = [eval(i) if isinstance(i, str) else i for i in top_gini_df.columns.tolist()]

#     # both eer_df and gini_df have same columns
#     # Convert list of tuples back to a MultiIndex
#     multiindex_columns = pd.MultiIndex.from_tuples(idx_tuples)
    
    gini_df.columns = multiindex_columns
    
    top_gini_df = gini_df.loc[eer_df.groupby([("", "type")])[("", "mean rank"),].idxmin()[('', 'mean rank')]]
    top_gini_df.insert(0, ("", "classifier"), clf_name)
    
    top_gini_df_lst.append(top_gini_df)

top_clf_eer_df = pd.concat(top_eer_df_lst)
top_clf_eer_df.drop(("", 'mean rank'), axis=1, inplace=True)
top_clf_eer_df.reset_index(drop=True, inplace=True)

top_clf_eer_df.insert(0, ("", 'mean rank'), top_clf_eer_df[("", "mean")].rank(method='min').astype(np.int64))
top_clf_eer_df.sort_values(("", 'mean rank'))

top_clf_gini_df = pd.concat(top_gini_df_lst)
top_clf_gini_df.drop(('', 'mean rank'), axis=1, inplace=True)
top_clf_gini_df.reset_index(drop=True, inplace=True)

top_clf_gini_df.insert(0, ("", 'mean rank'), top_clf_gini_df[("", "mean")].rank(method='min').astype(np.int64))
top_clf_gini_df.sort_values(("", 'mean rank'))

Window Size  \
   mean rank classifier cut_off_freq EMA_span SMA_winsize   type         125   
9          1         IF            -        -           -      -    0.335525   
11         2         IF           49        -           -   Real    0.329424   
10         3         IF            -       37           -  Naive    0.411922   
2          4      OCSVM           26        -           2   Real    0.517772   
0          5      OCSVM            -        -           -      -    0.505542   
3          6        kNN            -        -           -      -    0.500338   
5          7        kNN           33        -           -   Real    0.500188   
4          8        kNN            -       37           -  Naive    0.590468   
6          9        LOF            -        -           -      -    0.535434   
1         10      OCSVM           24       30           -  Naive    0.585105   
8         11        LOF           26        -           2   Real    0.536458   
7         12        LOF            -       49           -  Naive    0.641991   

                                                                          \
         250       500       750      1000      1250      1500      1750   
9   0.382443  0.422797  0.444283  0.481188  0.491695  0.505200  0.497006   
11  0.370514  0.415564  0.441483  0.490492  0.509331  0.527388  0.549604   
10  0.420017  0.447742  0.479385  0.517618  0.514364  0.514173  0.500893   
2   0.643776  0.751348  0.803055  0.809384  0.829675  0.845695  0.831326   
0   0.648686  0.760365  0.802483  0.824444  0.837377  0.846027  0.853184   
3   0.643844  0.750055  0.803336  0.823235  0.840662  0.852607  0.863799   
5   0.637842  0.756474  0.803453  0.822859  0.842396  0.852752  0.860681   
4   0.680762  0.758776  0.798135  0.818997  0.831698  0.839841  0.861540   
6   0.669907  0.768247  0.812912  0.834429  0.838725  0.854134  0.869947   
1   0.681948  0.771622  0.795331  0.830572  0.835501  0.848243  0.856070   
8   0.673346  0.776024  0.818321  0.830838  0.840478  0.861742  0.871662   
7   0.711383  0.774322  0.806974  0.833123  0.833132  0.851602  0.870412   

                                  
        2000      mean  variance  
9   0.533526  0.454852  0.004147  
11  0.558650  0.465828  0.006592  
10  0.521120  0.480804  0.001895  
2   0.843836  0.763985  0.012590  
0   0.854115  0.770247  0.014135  
3   0.876040  0.772657  0.015597  
5   0.878800  0.772827  0.015805  
4   0.871769  0.783554  0.008660  
6   0.884366  0.785345  0.012973  
1   0.869509  0.785989  0.008959  
8   0.875588  0.787162  0.012772  
7   0.873434  0.799597  0.006092

In [95]:
clf_df.style.format(STYLER_ERR_FORMAT_DICT).hide(axis='index').to_latex()

'\\begin{tabular}{rlllllrrrrrrrrrrr}\n\\multicolumn{6}{r}{} & \\multicolumn{9}{r}{Window Size} & \\multicolumn{2}{r}{} \\\\\nmean rank & classifier & cut_off_freq & EMA_span & SMA_winsize & type & 125 & 250 & 500 & 750 & 1000 & 1250 & 1500 & 1750 & 2000 & mean & variance \\\\\n8 & OCSVM & - & - & - & - & 0.1648 & 0.1075 & 0.0675 & 0.0597 & 0.0489 & 0.0485 & 0.0382 & 0.0396 & 0.0396 & 0.0683 & 0.0018 \\\\\n2 & OCSVM & 24 & 30 & - & Naive & 0.1508 & 0.0998 & 0.0646 & 0.0552 & 0.0431 & 0.0372 & 0.0330 & 0.0337 & 0.0365 & 0.0615 & 0.0016 \\\\\n6 & OCSVM & 26 & - & 2 & Real & 0.1619 & 0.0993 & 0.0649 & 0.0494 & 0.0459 & 0.0449 & 0.0369 & 0.0381 & 0.0429 & 0.0649 & 0.0017 \\\\\n9 & kNN & - & - & - & - & 0.1932 & 0.1138 & 0.0676 & 0.0542 & 0.0484 & 0.0422 & 0.0401 & 0.0366 & 0.0398 & 0.0707 & 0.0027 \\\\\n4 & kNN & - & 37 & - & Naive & 0.1514 & 0.1009 & 0.0666 & 0.0537 & 0.0449 & 0.0387 & 0.0337 & 0.0348 & 0.0369 & 0.0624 & 0.0016 \\\\\n7 & kNN & 33 & - & - & Real & 0.1814 & 0.1049 & 0.0680 &

In [96]:
df_lst=[]
for clf_name in CLASSIFIER_TYPE_LST:
    df = pd.read_json(f"{FINAL_EXP_RESULTS_PATH}/WACA-{clf_name}/WACA-{clf_name}-gini-df.json")
    
    idx_tuples = [eval(i) for i in df.columns.tolist()]

    # Convert list of tuples back to a MultiIndex
    multiindex_columns = pd.MultiIndex.from_tuples(idx_tuples)

    df.columns = multiindex_columns

    df = df.loc[df.groupby([("", "type")])[("", "mean rank"),].idxmin()[('', 'mean rank')]]
    df.insert(0, ("", "classifier"), clf_name)
    
    df_lst.append(df)

clf_df = pd.concat(df_lst)
clf_df.drop(("", 'mean rank'), axis=1, inplace=True)
clf_df.reset_index(drop=True, inplace=True)

clf_df.insert(0, ("", 'mean rank'), clf_df[("", "mean")].rank(method='min').astype(np.int64))
clf_df.sort_values(("", 'mean rank'))

Window Size  \
  mean rank classifier cut_off_freq EMA_span SMA_winsize   type         125   
2         1      OCSVM            -        9           -   Real    0.469574   
5         2        kNN            -        5           -   Real    0.487409   
4         3        kNN            -        -           9  Naive    0.502152   
0         4      OCSVM            -        -           -      -    0.505542   
1         5      OCSVM           41        -           -  Naive    0.519781   
3         6        kNN            -        -           -      -    0.500338   
8         7        LOF           33        -           -   Real    0.533809   
6         8        LOF            -        -           -      -    0.535434   
7         9        LOF           37        -           -  Naive    0.534079   

                                                                         \
        250       500       750      1000      1250      1500      1750   
2  0.605745  0.719450  0.790035  0.808972  0.838947  0.849311  0.858400   
5  0.620682  0.720804  0.792405  0.815536  0.822275  0.844888  0.872860   
4  0.628569  0.725530  0.787501  0.821889  0.825206  0.849391  0.855936   
0  0.648686  0.760365  0.802483  0.824444  0.837377  0.846027  0.853184   
1  0.648594  0.755737  0.798387  0.817226  0.829894  0.849773  0.856780   
3  0.643844  0.750055  0.803336  0.823235  0.840662  0.852607  0.863799   
8  0.670634  0.774432  0.820161  0.828985  0.847999  0.849145  0.862108   
6  0.669907  0.768247  0.812912  0.834429  0.838725  0.854134  0.869947   
7  0.676607  0.774012  0.823388  0.833569  0.850701  0.852439  0.867324   

                                 
       2000      mean  variance  
2  0.851103  0.754615  0.018127  
5  0.865935  0.760311  0.016804  
4  0.871460  0.763070  0.015415  
0  0.854115  0.770247  0.014135  
1  0.859871  0.770671  0.013268  
3  0.876040  0.772657  0.015597  
8  0.876067  0.784816  0.012723  
6  0.884366  0.785345  0.012973  
7  0.888227  0.788927  0.013103

In [97]:
clf_df.style.format(STYLER_ERR_FORMAT_DICT).hide(axis='index').to_latex()

'\\begin{tabular}{rlllllrrrrrrrrrrr}\n\\multicolumn{6}{r}{} & \\multicolumn{9}{r}{Window Size} & \\multicolumn{2}{r}{} \\\\\nmean rank & classifier & cut_off_freq & EMA_span & SMA_winsize & type & 125 & 250 & 500 & 750 & 1000 & 1250 & 1500 & 1750 & 2000 & mean & variance \\\\\n4 & OCSVM & - & - & - & - & 0.5055 & 0.6487 & 0.7604 & 0.8025 & 0.8244 & 0.8374 & 0.8460 & 0.8532 & 0.8541 & 0.7702 & 0.0141 \\\\\n5 & OCSVM & 41 & - & - & Naive & 0.5198 & 0.6486 & 0.7557 & 0.7984 & 0.8172 & 0.8299 & 0.8498 & 0.8568 & 0.8599 & 0.7707 & 0.0133 \\\\\n1 & OCSVM & - & 9 & - & Real & 0.4696 & 0.6057 & 0.7195 & 0.7900 & 0.8090 & 0.8389 & 0.8493 & 0.8584 & 0.8511 & 0.7546 & 0.0181 \\\\\n6 & kNN & - & - & - & - & 0.5003 & 0.6438 & 0.7501 & 0.8033 & 0.8232 & 0.8407 & 0.8526 & 0.8638 & 0.8760 & 0.7727 & 0.0156 \\\\\n3 & kNN & - & - & 9 & Naive & 0.5022 & 0.6286 & 0.7255 & 0.7875 & 0.8219 & 0.8252 & 0.8494 & 0.8559 & 0.8715 & 0.7631 & 0.0154 \\\\\n2 & kNN & - & 5 & - & Real & 0.4874 & 0.6207 & 0.7208 & 0.7

In [98]:
df_lst=[]
for clf_name in CLASSIFIER_TYPE_LST:
    df = pd.read_json(f"{FINAL_EXP_RESULTS_PATH}/WACA-{clf_name}/WACA-{clf_name}-relative_df.json")
    
    idx_tuples = [eval(i) for i in df.columns.tolist()]

    # Convert list of tuples back to a MultiIndex
    multiindex_columns = pd.MultiIndex.from_tuples(idx_tuples)

    df.columns = multiindex_columns

    df = df.loc[df.groupby([("", "type")])[("", "mean rank"),].idxmin()[('', 'mean rank')]]
    df.insert(0, ("", "classifier"), clf_name)
    
    df_lst.append(df)

clf_df = pd.concat(df_lst)
clf_df.drop(("", 'mean rank'), axis=1, inplace=True)
clf_df.reset_index(drop=True, inplace=True)

clf_df.insert(0, ("", 'mean rank'), clf_df[("", "mean")].rank(method='min').astype(np.int64))
relative_df=clf_df
relative_df

Window Size  \
  mean rank classifier cut_off_freq EMA_span SMA_winsize   type         125   
0         1      OCSVM            -        -           -      -    0.000000   
1         7      OCSVM           24       30           -  Naive    8.510028   
2         5      OCSVM           26        -           2   Real    1.733487   
3         1        kNN            -        -           -      -    0.000000   
4         9        kNN            -       37           -  Naive   21.618316   
5         6        kNN           33        -           -   Real    6.092150   
6         1        LOF            -        -           -      -    0.000000   
7         8        LOF            -       49           -  Naive   18.830051   
8         4        LOF           26        -           2   Real    2.961025   

                                                                               \
         250       500        750       1000       1250       1500       1750   
0   0.000000  0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
1   7.181751  4.387182   7.557812  11.835303  23.407301  13.587347  14.855515   
2   7.670572  3.885035  17.225042   6.222304   7.420663   3.235083   3.744404   
3   0.000000  0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
4  11.387103  1.485149   0.869565   7.182836   8.493151  15.897436   5.054945   
5   7.880382 -0.594059  11.242236   9.235075   9.178082  -2.564103   1.978022   
6   0.000000  0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
7  11.334274  6.237357   7.196030  11.489776   6.934813   5.222437   6.502242   
8   5.515145  3.000674  10.049628   7.594937   3.883495  -4.448743  10.986547   

                                   
       2000       mean   variance  
0  0.000000   0.000000   0.000000  
1  7.807387   9.851678  11.380259  
2 -8.415147   4.889658   4.672180  
3  0.000000   0.000000   0.000000  
4  7.209302  11.692142  41.405080  
5  0.697674   5.464044  13.385480  
6  0.000000   0.000000   0.000000  
7  7.692308  11.216713  37.399724  
8 -8.684864   3.811767   7.159257

In [99]:
clf_df.style.format(STYLER_ERR_FORMAT_DICT).hide(axis='index').to_latex()


'\\begin{tabular}{rlllllrrrrrrrrrrr}\n\\multicolumn{6}{r}{} & \\multicolumn{9}{r}{Window Size} & \\multicolumn{2}{r}{} \\\\\nmean rank & classifier & cut_off_freq & EMA_span & SMA_winsize & type & 125 & 250 & 500 & 750 & 1000 & 1250 & 1500 & 1750 & 2000 & mean & variance \\\\\n1 & OCSVM & - & - & - & - & 0.0000 & 0.0000 & 0.0000 & 0.0000 & 0.0000 & 0.0000 & 0.0000 & 0.0000 & 0.0000 & 0.0000 & 0.0000 \\\\\n7 & OCSVM & 24 & 30 & - & Naive & 8.5100 & 7.1818 & 4.3872 & 7.5578 & 11.8353 & 23.4073 & 13.5873 & 14.8555 & 7.8074 & 9.8517 & 11.3803 \\\\\n5 & OCSVM & 26 & - & 2 & Real & 1.7335 & 7.6706 & 3.8850 & 17.2250 & 6.2223 & 7.4207 & 3.2351 & 3.7444 & -8.4151 & 4.8897 & 4.6722 \\\\\n1 & kNN & - & - & - & - & 0.0000 & 0.0000 & 0.0000 & 0.0000 & 0.0000 & 0.0000 & 0.0000 & 0.0000 & 0.0000 & 0.0000 & 0.0000 \\\\\n9 & kNN & - & 37 & - & Naive & 21.6183 & 11.3871 & 1.4851 & 0.8696 & 7.1828 & 8.4932 & 15.8974 & 5.0549 & 7.2093 & 11.6921 & 41.4051 \\\\\n6 & kNN & 33 & - & - & Real & 6.0922 & 7.880

In [100]:
df_lst=[]
for clf_name in CLASSIFIER_TYPE_LST:
    df = pd.read_json(f"{FINAL_EXP_RESULTS_PATH}/WACA-{clf_name}/WACA-{clf_name}-gini-relative_df.json")
    
    idx_tuples = [eval(i) for i in df.columns.tolist()]

    # Convert list of tuples back to a MultiIndex
    multiindex_columns = pd.MultiIndex.from_tuples(idx_tuples)

    df.columns = multiindex_columns

    df = df.loc[df.groupby([("", "type")])[("", "mean rank"),].idxmin()[('', 'mean rank')]]
    df.insert(0, ("", "classifier"), clf_name)
    
    df_lst.append(df)

clf_df = pd.concat(df_lst)
clf_df.drop(("", 'mean rank'), axis=1, inplace=True)
clf_df.reset_index(drop=True, inplace=True)

clf_df.insert(0, ("", 'mean rank'), clf_df[("", "mean")].rank(method='min').astype(np.int64))
relative_df=clf_df
relative_df

Window Size  \
  mean rank classifier cut_off_freq EMA_span SMA_winsize   type         125   
0         3      OCSVM            -        -           -      -    0.000000   
1         2      OCSVM           41        -           -  Naive   -2.816425   
2         9      OCSVM            -        9           -   Real    7.114874   
3         3        kNN            -        -           -      -    0.000000   
4         7        kNN            -        -           9  Naive   -0.362544   
5         8        kNN            -        5           -   Real    2.584014   
6         3        LOF            -        -           -      -    0.000000   
7         1        LOF           37        -           -  Naive    0.253118   
8         6        LOF           33        -           -   Real    0.303433   

                                                                         \
        250       500       750      1000      1250      1500      1750   
0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
1  0.014224  0.608684  0.510452  0.875487  0.893608 -0.442735 -0.421486   
2  6.619684  5.380917  1.551242  1.876633 -0.187463 -0.388089 -0.611355   
3  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
4  2.372618  3.269702  1.971194  0.163460  1.838525  0.377171  0.910310   
5  3.597556  3.899816  1.360798  0.935206  2.187187  0.905302 -1.049002   
6  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
7 -1.000143 -0.750417 -1.288667  0.103090 -1.427922  0.198399  0.301563   
8 -0.108598 -0.805133 -0.891691  0.652498 -1.105765  0.584123  0.901154   

                                  
       2000      mean   variance  
0  0.000000  0.000000   0.000000  
1 -0.673902 -0.055080   6.135332  
2  0.352610  2.029466 -28.241524  
3  0.000000  0.000000   0.000000  
4  0.522894  1.240785   1.164411  
5  1.153518  1.597984  -7.739559  
6  0.000000  0.000000   0.000000  
7 -0.436568 -0.456197  -0.997914  
8  0.938373  0.067357   1.932993

In [101]:
relative_df.style.format(STYLER_IMPROVEMENT_FORMAT_DICT).hide(axis='index').to_latex()

'\\begin{tabular}{rlllllrrrrrrrrrrr}\n\\multicolumn{6}{r}{} & \\multicolumn{9}{r}{Window Size} & \\multicolumn{2}{r}{} \\\\\nmean rank & classifier & cut_off_freq & EMA_span & SMA_winsize & type & 125 & 250 & 500 & 750 & 1000 & 1250 & 1500 & 1750 & 2000 & mean & variance \\\\\n3 & OCSVM & - & - & - & - & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 \\\\\n2 & OCSVM & 41 & - & - & Naive & -2.82 & 0.01 & 0.61 & 0.51 & 0.88 & 0.89 & -0.44 & -0.42 & -0.67 & -0.06 & 6.14 \\\\\n9 & OCSVM & - & 9 & - & Real & 7.11 & 6.62 & 5.38 & 1.55 & 1.88 & -0.19 & -0.39 & -0.61 & 0.35 & 2.03 & -28.24 \\\\\n3 & kNN & - & - & - & - & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 \\\\\n7 & kNN & - & - & 9 & Naive & -0.36 & 2.37 & 3.27 & 1.97 & 0.16 & 1.84 & 0.38 & 0.91 & 0.52 & 1.24 & 1.16 \\\\\n8 & kNN & - & 5 & - & Real & 2.58 & 3.60 & 3.90 & 1.36 & 0.94 & 2.19 & 0.91 & -1.05 & 1.15 & 1.60 & -7.74 \\\\\n3 & LOF & - & - & - & - & 0.00 & 0.00 & 0.00 &